<a href="https://colab.research.google.com/github/alicelindel3/master/blob/main/Section_06/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# サポートベクターマシン
サポートベクターマシン（Support Vector Machine、SVM）とは、パターン識別のための教師あり機械学習の手法です。  
「マージン最大化」というアイディアに基づいているのですが、しばしば優れたパターン識別能力を発揮します。



## ●サポートベクターマシンとは？  
簡単にするために、2つの特徴量を持つデータを2つのグループに分類する絵を使います。

**（図: サポートベクターマシンの概念）**

サポートベクターマシンとは、グループを明確に分ける境界線を引くための手法です。  
こちらの図の例では、赤と青のクラスを明確に分ける境界線を引いています。  

この図で特徴量は2つ（２次元）なので境界は線になりますが、3次元の場合は境界は面になります。  
数学的に、直線や平面を一般化とした概念に「超平面」があります。  
線形サポートベクターマシンでは、この超平面を使ってn次元のデータの境界を定めます。  

上の図における直線の引き方ですが、「マージン最大化」により決定されます。  
この場合のマージンは、境界となる線からもっとも近い点との距離のことです。  
この場合は、赤と青それぞれのグループから線に最も近い2つずつの点のマージンを最大化するように線を引いています。  
このようなマージンの最大化に使われる境界付近の点を、サポートベクトルと呼びます。
マージンを最大化するために、赤のグループからも青のグループからももっとも遠い境界線を引くことになります。  

この境界線は「分類器」として機能し、新しいデータがどちらのグループに属するかを判別することができます。

## ●データセットの読み込み
今回は、scikit-learnに含まれるワインのデータセットを使用します。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine

wine = load_wine()
wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)  # data: 説明変数
wine_df["class"] = wine.target  # target: 目的変数
wine_df.head()

データセットの説明を表示します。

In [ ]:
print(wine.DESCR)

各統計量を表示します。

In [ ]:
wine_df.describe()

ライブラリseabornの`pairplot`により、説明変数同士、及び説明変数と目的変数の関係を一覧表示します。

In [ ]:
import seaborn as sns

sns.pairplot(wine_df, hue="class")

## ●SVMの実装
サポートベクターマシンを使い、ワインの分類を行います。  
まずは、データセットを訓練用のデータとテスト用のデータに分割し、標準化します。

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 訓練データとテストデータに分割
x_train, x_test, t_train, t_test = train_test_split(wine.data, wine.target, random_state=0) 

# データの標準化
std_scl = StandardScaler()
std_scl.fit(x_train)
x_train = std_scl.transform(x_train)
x_test = std_scl.transform(x_test)

今回は、線形サポートベクターマシンを使い、超平面によりデータを分類します。  

In [ ]:
from sklearn.svm import LinearSVC  # 線形ベクターマシン

model = LinearSVC(random_state=0)

# 全ての説明変数を使い学習
model.fit(x_train, t_train)

訓練済みのモデルを使い、訓練データ及びテストデータで予測を行います。  
そして、その正解率を測定します。  

In [ ]:
from sklearn.metrics import accuracy_score

# 予測結果
y_train = model.predict(x_train)
y_test = model.predict(x_test)
print(y_train, y_test)

# 正解率
acc_train = accuracy_score(t_train, y_train)
acc_test = accuracy_score(t_test, y_test)
print(acc_train, acc_test)

全てのデータのグループ分け結果を、グラフ表示します。

In [ ]:
import matplotlib.pyplot as plt

axis_1 = 0
axis_2 = 1

x = np.concatenate([x_train, x_test])
y = np.concatenate([y_train, y_test])
t = np.concatenate([t_train, t_test])

# 0にクラス分類されたグループ
group_0 = x[y==0]
plt.scatter(group_0[:, axis_1], group_0[:, axis_2], c="blue")

# 1にクラス分類されたグループ
group_1 = x[y==1]
plt.scatter(group_1[:, axis_1], group_1[:, axis_2], c="red")

# 2にクラス分類されたグループ
group_2 = x[y==2]
plt.scatter(group_2[:, axis_1], group_2[:, axis_2], c="green")

plt.xlabel(wine.feature_names[axis_1])
plt.ylabel(wine.feature_names[axis_2])
plt.show()

なお、結果はハイパーパラーメータを調整することで変化します。  
例えば、LinearSVCの引数として渡すことが可能なハイパーパラメータCは、正則化のためのパラメータです。  
この値を大きくすれば、誤分類によるペナルティが大きくなり、境界が複雑になります。  
ハイパーパラーメータについては、以下の公式ドキュメントを参考にしてください。  
https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html